# Method 2 (Equation 10 in the Nature review)

Here we show our second method of calculating the uncertainty of the free energy profile, which is actually an implementation of Equation 10 in Nature review paper: [Using metadynamics to explore complex free-energy landscapes](https://www.nature.com/articles/s42254-020-0153-0). Specifically, the uncertainty of the free energy surface can be estimated using the following equation:
$$\text{err}(s) = \frac{1}{\sqrt{N_{b}}}\sqrt{\text{Var}_{i}\left [ \log \left ( e^{\frac{B(s, t_{\text{fill}} + (i+1)L_{b})}{\Delta T}}-e^{\frac{B(s, t_{\text{fill}} + iL_{b})}{\Delta T}}\right )\right ]}$$

## 1. The code `calculate_free_energy.py`

To perform the analysis, we rely on the Python code `calculate_free_energy.py`, which takes in the arguments shown in the next cell. In summary, the input files required by this code include the PLUMED input file for running the metadynamics (`plumed.dat`), the PLUMED input file for reweighting (`plumed_reweight.dat`), and the HILLS file (`HILLS_LAMBDA`). The help message of `calculate_free_energy.py` is shown below:
```
usage: calculate_free_energy.py [-h] [-i INPUT] [-ir INPUT_RE] [-ih HILLS] -b
                                B_SIZES [B_SIZES ...] -p PERIODS [PERIODS ...]
                                [-dt DT] [-t TRUNCATION]

This code calculates the free energy profile and assess the influence of the
block size on its uncertainty.

optional arguments:
  -h, --help            show this help message and exit
  -i INPUT, --input INPUT
                        The PLUMED input file for running metadynamics.
                        Default: plumed.dat
  -ir INPUT_RE, --input_re INPUT_RE
                        The PLUMED input file for reweighting. Default:
                        plumed_reweight.dat
  -ih HILLS, --hills HILLS
                        The filename of the HILLS file. Default: HILLS*
  -b B_SIZES [B_SIZES ...], --b_sizes B_SIZES [B_SIZES ...]
                        The min, max and the spacing of the block sizes, in
                        the units of the STRIDE used to print COLVAR in
                        metadynamics. For example, if the this argument is set
                        to 5000 and STRIDE=10 was used in the PLUMED input
                        file for running the metadynamics, then the real block
                        size is 50000 simulation steps. If dt = 0.002 ps, this
                        means 100 ps. For this argument, if only one value is
                        given, only one free energy profile based on the given
                        value will be calcualted. (Only one fes_*.dat file.)
  -p PERIODS [PERIODS ...], --periods PERIODS [PERIODS ...]
                        The periods of the CV values. For example, if a
                        dihedral angle and a coordination number are biased in
                        a metadynamics, period = [2 * np.pi, 0]. Default: 0.
  -dt DT, --dt DT       The simulation time step (ps). Default: 0.002 ps.
  -t TRUNCATION, --truncation TRUNCATION
                        The truncation fraction. Default: 0.
```

Specifically, this code uses exactly the same method as Method 1 to calculate the free energy difference. The only difference between Method 1 and Method 2 lies in the estimation of the uncertainty. That is, identical to the steps in Method 1, `calculate_free_energy.py` first prepares a PLUMED input file `plumed_reweight.dat` (exactly the same as the one used in Method 1), runs the plumed driver to reweight the data and generate weighted histograms for each block. Then, it uses $F(s)=-kT\ln P(s)$ to calculate the free energy surface, hence the free energy difference. 

## 2. The command and results

To compare the results obtained from Method 2 to what we calculated in `Method_1.ipynb`, here we adopted 20 ps as our block size. As explained in the help message printed above, this means that we need to pass the value 1000 to use 10000 simulation steps as the block size. In addition, there is no periodicity for the alchemical variable, so we can run the following command to calculate the free energy surface: 
```
python calculate_free_energy.py -b 1000 -p 0
```
For the arguments that no values were specified, the default values were used. All the adopted parameters were documented in `parameters.yaml`. Note that the command took over 8 hours to run so we don't execute it here in the notebook again but just provide the results. 

As a result, with 20 ps (10000 simulation steps) as the block size, the free energy difference estimated by Method 2 was -3.200675 +/- 0.065225 kT. 

## 3. The issues with the method

Similar to what we presented in `Method_1.ipynb`, the first issue with this method is that it stills underestimate the true uncertainty. In addition, this method is extremely computationally expensive compared to Method 1 and the reason lies in the calculation of the cumulative biases $B(s, t_{\text{fill}} + (i+1)L_{b})$ and $B(s, t_{\text{fill}} + iL_{b})$, where $B(s, t_{\text{fill}} + (i+1)L_{b})$ is the total bias at CV being $s$ accumulated from $t_\text{fill}$ to the $(i+1)$-th block. To calculate this, we needed to read in the parameters of all the Gaussian deposited during the time interval of interest and sum up the values of all the Gaussians at CV being $s$. In our case, we have 250 blocks and 250000 Gaussians, which made the computation particularly expensive. (We could potentially use a much larger pace with a smaller bias factor, but the method itself is still expensive.) Overall, below are the questions that we are hoping to get help with:
- We wonder if our way of implementing Equation 10 in the paper is correct. If so, what could be the possible reasons that it underestimated the true uncertainty?
- Is there a faster way that we could accelerarte the calculation of the cumulative bias?

Thank you so much for you insights! You feedback has been useful to our project.